In [19]:
%pip install git+https://github.com/denis-spe/exttorch.git

  Cloning https://github.com/denis-spe/exttorch.git to /tmp/pip-req-build-qmod77y8
  Running command git clone --filter=blob:none --quiet https://github.com/denis-spe/exttorch.git /tmp/pip-req-build-qmod77y8
  Resolved https://github.com/denis-spe/exttorch.git to commit 535d92c10847058ace35731c1dc947716f52c48a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [31]:

import torch
import os
import sys
import numpy as np
# sys.path.append('../src/')
from exttorch.models import Sequential
from exttorch.callbacks import EarlyStopping
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torch.optim import Adam
from exttorch.metrics import Accuracy
import pandas as pd
from exttorch._data_handle import DataHandler

In [21]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
    train_df = pd.read_csv('./drive/MyDrive/Datasets/MNIST/train.csv')
    test_df = pd.read_csv('./drive/MyDrive/Datasets/MNIST/test.csv')
except:
    train_df = pd.read_csv('/Volumes/Storage/DS/DL/exttorch/datasets/digit-recognizer/train.csv')
    test_df = pd.read_csv('/Volumes/Storage/DS/DL/exttorch/datasets/digit-recognizer/test.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
mask = np.random.randn(len(train_df)) <= 0.8
train_data = train_df[mask]
valid_data = train_df[~mask]
print(f"Train_data Shape {train_data.shape}\nValid_data: {valid_data.shape}")

Train_data Shape (32998, 785)
Valid_data: (9002, 785)


In [23]:
class CreateDataset(Dataset):
    def __init__(self, df: pd.DataFrame, transforms: transforms = None,
                 train: bool =True, normalize: bool = True):
        self.transforms = transforms
        self.train = train
        self.df = df
        self.len = self.df.shape[0]
        self.normalize = normalize

    def __len__(self):
        return self.len

    def __getitem__(self, index: int):
        if 'label' in self.df.columns:
            image = self.df.iloc[index, 1:].values.reshape(28, 28)
            if self.normalize:
                image = self.df.iloc[index, 1:].values.reshape(28, 28) / 255
            label = self.df.iloc[index, 0]
            if self.transforms:
                image = self.transforms(image).type(torch.float)
            return image, label
        else:
            image = self.df.iloc[index, :].values.reshape(28, 28)
            if self.normalize:
                image = self.df.iloc[index, :].values.reshape(28, 28) / 255
            if self.transforms:
                image = self.transforms(image).type(torch.float)
            return image

In [24]:
# Construct Datasets train_X_set
train_X_set = CreateDataset(
    train_data,
    transforms=transforms.Compose([transforms.ToTensor()]))

# Construct valid_X_set dataset
valid_X_set = CreateDataset(
    valid_data,
    transforms=transforms.Compose([transforms.ToTensor()]))

# Construct test_data
test_data = CreateDataset(
    test_df,
    transforms=transforms.Compose([transforms.ToTensor()]))

In [25]:
BATCH_SIZE=64

In [26]:
# Coupling Datasets with dataloader function
train_dataloader = DataLoader(train_X_set, batch_size=BATCH_SIZE, shuffle=True)
valid_dataloader = DataLoader(valid_X_set, batch_size=BATCH_SIZE, shuffle=True)

test_dataloader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False)

In [27]:
model = Sequential([
    # Transpose Input data
    nn.Flatten(),

    # Input layer
    nn.Linear(in_features=28 * 28, out_features=256),
    nn.ReLU(),  # Activation function
    nn.Dropout(.4),  # Drop same pixel

    nn.Linear(in_features=256, out_features=256),
    nn.ReLU(),  # Activation function
    nn.Dropout(.4),  # Drop same pixel

    # Output layer
    nn.Linear(in_features=256, out_features=10),
    nn.Softmax(dim=1)
])



In [32]:

model.compile(
    optimizer=Adam(model.parameters()),
    loss=nn.CrossEntropyLoss(),
    metrics=[Accuracy()]
)

history = model.fit(
    train_dataloader,
    epochs=120,
    validation_data=valid_dataloader,
    callbacks=[EarlyStopping(patience=3)]
    )


Epoch 1/120
517/516 ━━━━━━━━━━━━━━━━━━━━ 23s 44ms/step - loss: 1.4987 - Accuracy: 0.9631 - val_loss: 1.4970 - val_Accuracy: 0.9647
Epoch 2/120
517/516 ━━━━━━━━━━━━━━━━━━━━ 22s 42ms/step - loss: 1.5003 - Accuracy: 0.9606 - val_loss: 1.4958 - val_Accuracy: 0.9659
Epoch 3/120
517/516 ━━━━━━━━━━━━━━━━━━━━ 24s 47ms/step - loss: 1.4983 - Accuracy: 0.9635 - val_loss: 1.4929 - val_Accuracy: 0.9689
Epoch 4/120
517/516 ━━━━━━━━━━━━━━━━━━━━ 22s 43ms/step - loss: 1.4965 - Accuracy: 0.9651 - val_loss: 1.4953 - val_Accuracy: 0.9658
Epoch 5/120
517/516 ━━━━━━━━━━━━━━━━━━━━ 22s 42ms/step - loss: 1.4954 - Accuracy: 0.9658 - val_loss: 1.4926 - val_Accuracy: 0.9690
Epoch 6/120
517/516 ━━━━━━━━━━━━━━━━━━━━ 23s 44ms/step - loss: 1.4937 - Accuracy: 0.9679 - val_loss: 1.4951 - val_Accuracy: 0.9658
Epoch 7/120
517/516 ━━━━━━━━━━━━━━━━━━━━ 23s 44ms/step - loss: 1.4943 - Accuracy: 0.9672 - val_loss: 1.4937 - val_Accuracy: 0.9679
Epoch 8/120
517/516 ━━━━━━━━━━━━━━━━━━━━ 23s 44ms/step - loss: 1.4933 - Accuracy: 

In [33]:
history.history

{'Accuracy': [0.9631,
  0.9606,
  0.9635,
  0.9651,
  0.9658,
  0.9679,
  0.9672,
  0.9677,
  0.9677,
  0.9682,
  0.9689,
  0.9689,
  0.9698,
  0.9691,
  0.9697,
  0.9708,
  0.9717,
  0.9711,
  0.9697,
  0.9715],
 'val_Accuracy': [0.9647,
  0.9659,
  0.9689,
  0.9658,
  0.969,
  0.9658,
  0.9679,
  0.9697,
  0.9678,
  0.9678,
  0.9688,
  0.9669,
  0.9709,
  0.9688,
  0.9709,
  0.9715,
  0.9728,
  0.9726,
  0.972,
  0.9714],
 'loss': [1.4986,
  1.5003,
  1.4983,
  1.4965,
  1.4955,
  1.4937,
  1.4943,
  1.4932,
  1.4936,
  1.4927,
  1.4922,
  1.4922,
  1.4913,
  1.4918,
  1.4915,
  1.4902,
  1.4896,
  1.4902,
  1.4914,
  1.4897],
 'val_loss': [1.497,
  1.4958,
  1.4929,
  1.4953,
  1.4926,
  1.4951,
  1.4937,
  1.4914,
  1.4937,
  1.4933,
  1.4925,
  1.4942,
  1.4905,
  1.4922,
  1.4902,
  1.4898,
  1.4882,
  1.4886,
  1.4895,
  1.49]}